Nộp lại yêu cầu 1 , yêu cầu 2 đã failed

Import package

In [2]:
from pyspark.sql import SparkSession , Window 
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, IntegerType, ArrayType
from pyspark.sql import functions as f


Đọc dữ liệu từ MongoDB với Spark

In [3]:
spark_Questions=SparkSession \
    .builder \
    .master('local') \
    .appName('MyApp') \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') \
    .config('spark.mongodb.input.uri', 'mongodb://127.0.0.1/dep_303.Questions') \
    .config('spark.mongodb.output.uri', 'mongodb://127.0.0.1/dep_303') \
    .getOrCreate()
kafka_Questions_df = spark_Questions.read \
    .format('com.mongodb.spark.sql.DefaultSource') \
    .option('uri', 'mongodb://127.0.0.1/dep_303.Questions') \
    .load()

spark_answer=SparkSession \
    .builder \
    .master('local') \
    .appName('MyApp') \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') \
    .config('spark.mongodb.input.uri', 'mongodb://127.0.0.1/dep_303.Answers') \
    .config('spark.mongodb.output.uri', 'mongodb://127.0.0.1/dep_303') \
    .getOrCreate()
kafka_answer_df = spark_answer.read \
    .option('uri', 'mongodb://127.0.0.1/dep_303.Answers') \
    .format('com.mongodb.spark.sql.DefaultSource') \
    .load()

In [ ]:
kafka_Questions_df.show(5)
kafka_answer_df.show(5)

Chuẩn hóa dữ liệu Question

In [4]:
#Chuẩn hóa dữ liệu
Questions_df =kafka_Questions_df.withColumn("CreationDate",to_date(col("CreationDate"),"yyyy-MM-dd'T'HH:mm:ss'Z'")) \
                                .withColumn("ClosedDate",to_date(col("ClosedDate"),"yyyy-MM-dd'T'HH:mm:ss'Z'")) \
                                .withColumn("OwnerUserId",kafka_Questions_df["OwnerUserId"].cast(IntegerType()))
Questions_df.printSchema()
Questions_df.show()

root
 |-- Body: string (nullable = true)
 |-- ClosedDate: date (nullable = true)
 |-- CreationDate: date (nullable = true)
 |-- Id: integer (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)

+--------------------+----------+------------+----+-----------+-----+--------------------+--------------------+
|                Body|ClosedDate|CreationDate|  Id|OwnerUserId|Score|               Title|                 _id|
+--------------------+----------+------------+----+-----------+-----+--------------------+--------------------+
|<p>I've written a...|      null|  2008-08-01|  80|         26|   26|SQLStatement.exec...|{62e28195c5852c9c...|
|<p>Has anyone got...|      null|  2008-08-01| 120|         83|   21|   ASP.NET Site Maps|{62e28195c5852c9c...|
|<p>Are there any ...|2012-12-26|  2008-08-01|  90|         58|  144|Good branching an.

Chuẩn hoá dữ liệu cho Answer

In [ ]:
#Chuẩn hóa dữ liệu
Answers_df =kafka_answer_df.withColumn("CreationDate",to_date(col("CreationDate"),"yyyy-MM-dd'T'HH:mm:ss'Z'")) \
                                .withColumn("OwnerUserId",kafka_answer_df["OwnerUserId"].cast(IntegerType()))
Answers_df.printSchema()
Answers_df.show()

Yêu cầu 1: Tính số lần xuất hiện của các ngôn ngữ lập trình

In [6]:
#regex Programing Language
r = 'Java|Python|[C][+][+]|C#|Go|Ruby|PHP|HTML|CSS|SQL'

In [66]:
#tìm tất cả các ngôn ngữ
Yeu_cau_1=kafka_Questions_df.withColumn("Body", f.expr(f"regexp_extract_all(Body, '{r}', 0)"))
#đưa các ngôn ngữ thành từng row
Yeu_cau_1 = Yeu_cau_1.select(f.explode('Body').alias("Programing Language"))
#group >> output
Yeu_cau_1.groupBy("Programing Language").agg(f.count("Programing Language").alias("count")).show()

+-------------------+------+
|Programing Language| count|
+-------------------+------+
|                 C#| 32414|
|                C++| 28866|
|                CSS| 33556|
|               HTML| 89646|
|                PHP| 63479|
|                SQL|146094|
|                 Go| 79912|
|               Ruby| 16318|
|             Python| 44817|
|               Java|106659|
+-------------------+------+



Yêu cầu 2 : Tìm các domain được sử dụng nhiều nhất trong các câu hỏi

In [5]:
#regex domain
r_2='(?:[a-z0-9](?:[a-z0-9-]{0,61}[a-z0-9])?\.)+[a-z0-9][a-z0-9-]{0,61}[a-z0-9]'

In [ ]:
#lấy tất cả các domain trong row
a1 =kafka_Questions_df.withColumn("Body", f.expr(f"regexp_extract_all(Body, '{r_2}', 0)"))
#đưa Domain thành từng row
a2 = a1.select(f.explode('Body').alias('Domain'))
#group >>  output
a3 =a2.groupBy("Domain").agg(f.count("Domain").alias("count")).show()

In [ ]:
#Yeu_cau_2= kafka_Questions_df.withColumn("Body" , f.regexp_extract("Body",r_2, 0))
#Yeu_cau_2.groupBy("Body").agg(f.count("Body").alias("count")).show()

Yêu cầu 3 : Tính tổng điểm của User theo từng ngày

In [ ]:
Yeu_cau_3 = Questions_df.groupBy("OwnerUserId","CreationDate").agg(f.sum("Score").alias("TotalScore")).show()

+-----------+------------+----------+
|OwnerUserId|CreationDate|TotalScore|
+-----------+------------+----------+
|       1464|  2008-08-19|        38|
|       9947|  2008-09-15|         7|
|       1220|  2008-09-18|         1|
|      18010|  2008-09-18|         6|
|         45|  2008-08-05|        12|
|        509|  2008-08-12|         0|
|        522|  2008-08-22|         4|
|        946|  2008-09-02|       978|
|       4257|  2008-09-11|         2|
|       8344|  2008-09-15|        42|
|        832|  2008-09-03|         6|
|       8446|  2008-09-16|        34|
|       1848|  2008-08-18|        26|
|       1560|  2008-08-19|       130|
|       1693|  2008-08-25|         7|
|       2515|  2008-09-02|         3|
|        305|  2008-09-02|         9|
|       3225|  2008-09-12|         8|
|       2755|  2008-09-18|        32|
|        466|  2008-08-22|         4|
+-----------+------------+----------+
only showing top 20 rows



Yêu cầu 4: Tính tổng số điểm mà User đạt được trong một khoảng thời gian

In [ ]:
#cửa sổ trược
running_total_window = Window.partitionBy("OwnerUserId") \
        .orderBy("CreationDate") \
        .rowsBetween(Window.unboundedPreceding, Window.currentRow)

Yeu_cau_4 = Questions_df.withColumn("TotalScore",
                          f.sum("Score").over(running_total_window)) 

Yeu_cau_4.select("OwnerUserId","CreationDate","TotalScore").orderBy("CreationDate").show()

+-----------+------------+----------+
|OwnerUserId|CreationDate|TotalScore|
+-----------+------------+----------+
|         83|  2008-08-01|        21|
|         26|  2008-08-01|        26|
|         91|  2008-08-01|        49|
|         58|  2008-08-01|       144|
|    2089740|  2008-08-01|        53|
|         71|  2008-08-02|        13|
|         63|  2008-08-02|        29|
|         91|  2008-08-02|        70|
|        143|  2008-08-03|        79|
|        233|  2008-08-03|         9|
|         60|  2008-08-04|        18|
|         67|  2008-08-04|        14|
|        245|  2008-08-04|        28|
|        236|  2008-08-04|        17|
|         91|  2008-08-04|        93|
|        120|  2008-08-04|        36|
|        254|  2008-08-04|        42|
|        328|  2008-08-04|        63|
|        230|  2008-08-04|        18|
|        281|  2008-08-04|        17|
+-----------+------------+----------+
only showing top 20 rows



 Yêu cầu 5: Tìm các câu hỏi có nhiều câu trả lời

In [ ]:
#làm quen với join
#join 2 datafame
join_expr = Questions_df.Id == Answers_df.ParentId

Yeu_cau_5 = Questions_df.join(Answers_df, join_expr, "inner").groupBy("ParentId").agg(f.count("ParentId").alias("Total"))

In [ ]:
# Total đang kiểu dữ liệu bigint nên cần chuyển về int
Yeu_cau_5.withColumn("Total",Yeu_cau_5["Total"].cast(IntegerType()))
Yeu_cau_5.filter(col("Total") >= 5).orderBy("ParentId").show()

+--------+-----+
|ParentId|Total|
+--------+-----+
|     180|    9|
|     260|    9|
|     330|   10|
|     580|   14|
|     650|    6|
|     930|    7|
|    1040|    7|
|    1160|   12|
|    1300|    7|
|    1390|    6|
|    1600|    5|
|    1610|    8|
|    1760|   12|
|    1970|    8|
|    2120|    7|
|    2250|    5|
|    2300|    6|
|    2530|   38|
|    2540|    5|
|    2550|    7|
+--------+-----+
only showing top 20 rows



In [ ]:
a = Answers_df.groupBy("ParentId").agg(f.count("ParentId").alias("Total"))
a.filter(col("Total") >= 5).orderBy("ParentId").show()

+--------+-----+
|ParentId|ToTal|
+--------+-----+
|     180|    9|
|     260|    9|
|     330|   10|
|     580|   14|
|     650|    6|
|     930|    7|
|    1040|    7|
|    1160|   12|
|    1300|    7|
|    1390|    6|
|    1600|    5|
|    1610|    8|
|    1760|   12|
|    1970|    8|
|    2120|    7|
|    2250|    5|
|    2300|    6|
|    2530|   38|
|    2540|    5|
|    2550|    7|
+--------+-----+
only showing top 20 rows



(Nâng cao) Yêu cầu 6: Tìm các Active User

In [ ]:
#đổi tên cột cho dể làm việc
product_question_df = Questions_df.withColumnRenamed("CreationDate","CreationDate_question") \
    .withColumnRenamed("OwnerUserId", "OwnerUserId_question") \
    .withColumnRenamed("Score", "Score_question") \
    .withColumnRenamed("Id","Id_question")
product_answer_df = Answers_df.withColumnRenamed("Id","Id_answer") \
    .withColumnRenamed("CreationDate" , "CreationDate_answer")

In [ ]:
#join 2 colection QUestion và Answer
join_expr_6 = product_question_df.Id_question == product_answer_df.ParentId
Join_6 = product_question_df.join(product_answer_df, join_expr_6, "inner") \
    .selectExpr("Id_question","Id_answer","CreationDate_question","CreationDate_answer","Score_question","OwnerUserId_question")

In [ ]:
User_1 = Join_6.groupBy("Id_question" ,"CreationDate_question" ,"Score_question","OwnerUserId_question" ) \
    .agg(f.count("Id_question").alias("Total"))

# Total đang kiểu dữ liệu bigint nên cần chuyển về int
User_1.withColumn('Total',User_1['Total'].cast(IntegerType()))

DataFrame[Id_question: int, CreationDate_question: date, Score_question: int, OwnerUserId_question: int, Total: int]

vấn đề này em chia 3 yêu cầu filter thành 3 datafame . sau đó dùng Full outer join 3 datafame đó để có được ID_questtion . Cuối cùng em inner join giữa ID_question và Question_df để lấy được User

In [ ]:
User_1 = User_1.filter(col("Total") >=50).selectExpr("Id_question")
User_1.show()

+-----------+
|Id_question|
+-----------+
|      23930|
|      38210|
|      40480|
|     100420|
|     106340|
|     202750|
|     226970|
|     406760|
|     490420|
|    1218390|
|    2155930|
+-----------+



In [ ]:
User_2 = Answers_df.groupBy("ParentId").agg(f.sum("Score").alias("sum"))
User_2 = User_2.filter(col("sum") >= 500).selectExpr("ParentId")
User_2.show()

+--------+
|ParentId|
+--------+
|   32260|
|   23250|
|   23930|
|   34510|
|   18450|
|    9410|
|   11620|
|    2530|
|   16100|
|  100420|
| 1100260|
| 1283960|
| 1459750|
|   65820|
|  406230|
|  894860|
|  105770|
| 1466000|
| 1960240|
|  490420|
+--------+
only showing top 20 rows



In [ ]:
User_3 = Join_6.groupBy("Id_question", "CreationDate_answer").agg(f.count("CreationDate_answer").alias("Total"))
# Total đang kiểu dữ liệu bigint nên cần chuyển về int
User_3.withColumn('Total',User_3['Total'].cast(IntegerType()))
User_3 = User_3.filter(col("Total") > 5).selectExpr("Id_question")
User_3.show()

+-----------+
|Id_question|
+-----------+
|       1160|
|       3400|
|       4880|
|       6080|
|       8050|
|       8800|
|       9240|
|       9750|
|      10230|
|      10260|
|      10860|
|      11330|
|      11500|
|      11620|
|      12870|
|      13000|
|      15690|
|      16460|
|      16550|
|      16860|
+-----------+
only showing top 20 rows



In [ ]:
#full outer join
join_expr_full_1 = User_1.Id_question == User_2.ParentId
User_12 = User_1.join(User_2,join_expr_full_1,"outer").select("ParentId")

In [ ]:
join_expr_full_2 = User_12.ParentId == User_3.Id_question
User_123 = User_12.join(User_3,join_expr_full_2,"outer").select("*").show()